# Train a domain classifier on the [semantic scholar dataset](https://api.semanticscholar.org/corpus)

## Part 3: Create production inference function

In the [previous notebook](train.ipynb), we trained our AI model. Now, it's time to create **G**eneral **R**obust **E**nd-to-end **A**utomated **T**rustworthy deployment from it using the `GreatAI` Python package.

In [1]:
from typing import List
import re
from great_ai import ClassificationOutput, GreatAI, use_model
from sklearn.pipeline import Pipeline
from great_ai.utilities.clean import clean

In [2]:
@GreatAI.deploy
@use_model("small-domain-prediction-v2", version="latest")
def predict_domain(
    text: str, model: Pipeline, target_confidence: int = 20
) -> List[ClassificationOutput]:
    """
    Predict the scientific domain of the input text.
    Return labels until their sum likelihood is larger than target_confidence.
    """
    assert 0 <= target_confidence <= 100, "invalid argument"

    preprocessed = re.sub(r"[^a-zA-Z ]", "", clean(text, convert_to_ascii=True))
    features = model.named_steps["vectorizer"].transform([preprocessed])
    prediction = model.named_steps["classifier"].predict_proba(features)[0]

    best_classes = sorted(enumerate(prediction), key=lambda v: v[1], reverse=True)

    results: List[ClassificationOutput] = []
    for class_index, probability in best_classes:
        results.append(
            ClassificationOutput(
                label=model.named_steps["classifier"].classes_[class_index],
                confidence=round(probability * 100),
                explanation=[
                    word
                    for _, word in sorted(
                        (
                            (weight, word)
                            for weight, word, count in zip(
                                model.named_steps["classifier"].feature_log_prob_[
                                    class_index
                                ],
                                model.named_steps["vectorizer"].get_feature_names_out(),
                                features.A[0],
                            )
                            if count > 0
                        ),
                        reverse=True,
                    )
                ][:5],
            )
        )

        if sum(r.confidence for r in results) >= target_confidence:
            break

    return results

2022-05-28 15:02:20,852 |  WARNING | Environment variable ENVIRONMENT is not set, defaulting to development mode ‼️
2022-05-28 15:02:20,853 |     INFO | Options: configured ✅
2022-05-28 15:02:21,168 |     INFO | Latest version of small-domain-prediction-v2 is 8 (from versions: 3, 4, 5, 6, 7, 8)
2022-05-28 15:02:21,169 |     INFO | File small-domain-prediction-v2-8 found in cache


In [3]:
result = predict_domain("""
    State-of-the-art methods for zero-shot visual recognition formulate learning as a joint embedding problem of images and side information. In these formulations the current best complement to visual features are attributes: manually encoded vectors describing shared characteristics among categories. Despite good performance, attributes have limitations: (1) finer-grained recognition requires commensurately more, and (2) attributes do not provide a natural language interface. We propose to overcome these limitations by training neural language models from scratch; i.e. without pre-training and only consuming words and characters. Our proposed models train end-to-end to align with the fine-grained and category-specific content of images. Natural language provides a flexible and compact way of encoding only the salient visual aspects for distinguishing categories. By training on raw text, our model can do inference on raw text as well, providing humans a familiar mode both for annotation and retrieval. Our model achieves strong performance on zero-shot text-based image retrieval and significantly outperforms the attribute-based state-of-the-art for zero-shot classification on the CaltechUCSD Birds 200-2011 dataset. """
)

from pprint import pprint
pprint(result.dict(), width=120)

{'created': '2022-05-28T13:02:22.035886',
 'evaluation': None,
 'evaluation_id': 'c035d78e-dea9-415f-8f93-0dc4cd8dd7b5',
 'exception': None,
 'execution_time_ms': 94.814,
 'logged_values': {'arg:predict_domain:target_confidence': 20,
                   'arg:predict_domain:text': '\n'
                                              '    State-of-the-art methods for zero-shot visual recognition formulate '
                                              'learning as a joint embedding problem of images and side information. '
                                              'In these formulations the current best complement to visual features '
                                              'are attributes: manually encoded vectors describing shared '
                                              'characteristics among categories. Despite good performance, attributes '
                                              'have limitations: (1) finer-grained recognition requires commensurately '
        